# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up



In [1]:
# Do all imports and installs here


import os
import sys
import boto3

import datetime
import numpy as np
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 2)


# Step 1: Scope the Project and Gather Data

#### Scope TODO
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data TODO
Describe the data sets you're using. Where did it come from? What type of information is included? 

# Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [51]:
# Read in the data here
!ls -1FSash ./dataset
#!ls -tRFh ./dataset/
path = './dataset/'

total 1,2G
510M airports_us.csv
509M GlobalLandTemperaturesByCity.csv
205M WDIData.csv
5,8M airport-codes_csv.csv
248K us-cities-demographics.csv
144K immigration_data_sample.csv
 36K I94_SAS_Labels_Descriptions.SAS
4,0K ./
4,0K ../
4,0K 20-years-us-university-dataset/
4,0K airline-delay-and-cancellation-data-2009-2018/
4,0K education-statistics/
4,0K sas_data/


### Immigration data

 * U.S. immigration officers
 * I-94 Form (Arrival/Departure Record) to foreign visitors 
 * Explanation about the Visitor Arrivals Program (I-94 Form) [here](https://travel.trade.gov/research/programs/i94/description.asp)
 * data about arrival and why and during
 * dataset = 1 file per month
 * here: april 2026

TODO
* immigration_data_sample
    * revoir l'orignine du fichier. chercher data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat(pd.read_sas(immigration_fname, 'sas7bdat', encoding="ISO-8859-1")
    ==> 1000 rows fourni par udac, beaucoup plus dans 1 seul mois
    * faire un dictionnaire, recuperer les colonnes. 

In [81]:
nRowsRead = None # change and set to None for the whole data

In [87]:
description    = "dataset immigration provide by Udacity"
name           = "df_immigration"
file           = "immigration_data_sample.csv"
n_df           = pd.read_csv(path+file, nrows = nRowsRead)
nRow, nCol     = n_df.shape
print("There are {} rows and {} columns in {}.".format(nRow, nCol, description))
df_immigration = n_df
#print(df_immigration.head(1))
#print(df_immigration.info())


#dic_6 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}

There are 1000 rows and 29 columns in dataset immigration provide by Udacity.


##### Data Dictionary

Column Name | Description | Example | Type
-|-|-|-|
cicid|     ID uniq per record in the dataset | 4.08e+06 | float64
i94yr|     4 digit year  | 2016.0 | float64
i94mon|    Numeric month |  4.0 | float64
i94ci|     3 digit code of source city for immigration (Born country) | 209.0 | float64
i94res|    3 digit code of source country for immigration (Residence country) | 209.0 | float64
i94port|   Port addmitted through | HHW | object
arrdate|   Arrival date in the USA | 20566.0 | float64
i94mode|   Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported) | 1.0 | float
i94addr|   State of arrival | HI | object
depdate|   Departure date | 20573.0 | float
i94bir|    Age in years | 61.0 | float
i94visa|   Visa Code - 1 = Business / 2 = Pleasure / 3 = Student |2.0 | float
count|     Used for summary statistics | 1.0 | float
dtadfile|  Character Date Field |20160422| int 64 
visapost|  Department of State where where Visa was issued | | object
occup|     Occupation that will be performed in U.S. || object
entdepa|   Arrival Flag - Whether admitted or paroled into the US |G| object
entdepd|   Departure Flag. Whether departed, lost visa, or deceased |O|  object
entdepu|   Update Flag - Either apprehended, overstayed, adjusted to perm residence || float64
matflag|   Match flag |M|  object
biryear|   4 digit year of birth |1955.0| float64
dtaddto|   Character date field to when admitted in the US |07202016| object
gender|    Gender|M| object
insnum|    INS number || float64
airline|   Airline used to arrive in U.S.|JL| object
admnum|    Admission number, should be unique and not nullable |5.66e+10| float
fltno|     Flight number of Airline used to arrive in U.S. |00782| object
visatype|  Class of admission legally admitting the non-immigrant to temporarily stay in U.S.|WT|object

In [83]:
# airports_us.csv KAGGLE
description   = " Dataset from KAGGLE, usefule to follow where go aliens"
name          = "df_airport_us"
file          = "airports_us.csv"

n_df          = pd.read_csv(path+file, nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, file))
df_airport_us = n_df
df_airport_us.head()

#dic_1 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_1

There are 3606803 rows and 15 columns in DataFrame airports_us.csv.


,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.14,-96.67,NaN,NaN
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.12,-123.21,44.25,-121.15
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.12,-123.21,44.25,-121.15
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.12,-123.21,44.25,-121.15
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.37,-122.87,44.25,-121.15


In [68]:
# download from kaggle the GlobalLandTemperaturesByCity.csv KAGGLE/UDACITY
description   = "download from kaggle the GlobalLandTemperaturesByCity.csv KAGGLE/UDACITY"
name          = "df_temp"
file          = "GlobalLandTemperaturesByCity.csv"

n_df          = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_temp       = n_df

dic_2 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_2

There are 8599212 rows and 7 columns in DataFrame df_temp.


In [7]:
# WDIData.csv Indicators developpement KAGGLE
description      = "WDIData.csv country Indicators developpment KAGGLE"
name             = "df_indicator_dev"
file             = "WDIData.csv"

n_df             = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol       = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_indicator_dev = n_df

dic_3 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
# dic_3

There are 1000 rows and 64 columns in DataFrame df_indicator_dev.


In [8]:
# airport-codes_csv UDACITY
description      = "airport-codes_csv provide by UDACITY"
name             = "df_airport_world"
file             = "airport-codes_csv.csv"

n_df             = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol       = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_airport_world = n_df

dic_4 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_4

There are 1000 rows and 12 columns in DataFrame df_airport_world.


In [9]:
# us-cities-demographics USACITY
description   = "us-cities-demographics provide by UDACITY"
name          = "df_demograph"
file          = "us-cities-demographics.csv"

n_df          = pd.read_csv(path+file, sep=";", nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_demograph  = n_df

dic_5 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_5

There are 1000 rows and 12 columns in DataFrame df_demograph.


In [11]:
#I94 DESCRIPTION
description   = "I94_SAS_Labels_Descriptions.SAS"
name          = "df_label_I94"
file          = "I94_SAS_Labels_Descriptions.SAS"

n_df          = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_<>       = n_df

dic_<> = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
dic_<>


SyntaxError: invalid syntax (<ipython-input-11-fe2b6f915f96>, line 2)

In [12]:
#df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [13]:
# SAS
description   = " Some parquet files in directory 'sas_data'"
name          = "df_sas"
file          = "./dataset/sas_data"

n_df          = spark.read.parquet("./dataset/sas_data")
nRow, nCol    = n_df.count(), len(n_df.columns)
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_sas        = n_df

dic_8 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_8

There are 3096313 rows and 28 columns in DataFrame df_sas.


In [14]:
# ./dataset/20-years-us-university-dataset
# decide to not use it for now. 

In [28]:
# ./dataset/airline-delay-and-cancellation-data-2009-2018/2016.csv
description   = ".Data about us flight in 2016"
name          = "df_airline_delay"
file          = "airline-delay-and-cancellation-data-2009-2018/2016.csv"

n_df          = pd.read_csv(path+file, nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_airline_delay       = n_df

dic_10 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_10
type(dic_10)

There are 1000 rows and 28 columns in DataFrame df_airline_delay.


dict

In [16]:
list_dic = [dic_1, dic_2, dic_3, dic_4, dic_5, dic_6, dic_8, dic_9, dic_10]

description   =
name          =
file          =

n_df          = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_temp       = n_df

dic_n = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
dic_n

SyntaxError: invalid syntax (<ipython-input-16-30f9d1c4f2db>, line 3)

In [17]:
# /dataset/education-statistics 

description           = "Data from education-statistics"
name                  = "df_Educ_country_series"
file                  = "education-statistics/EdStatsCountry-Series.csv"
n_df                  = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol            = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_Educ_country_series = n_df
dic_11 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_11

description   = "Data from education-statistics"
name          = "df_Educ_country"
file          = "education-statistics/EdStatsCountry.csv"
n_df          = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_Educ_country = n_df
dic_12 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_12

description   = "Data from education-statistics"
name          = "df_Educ_data"
file          = "education-statistics/EdStatsData.csv"
n_df          = pd.read_csv(path+file, nrows = nRowsRead)
nRow, nCol    = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_Educ_data  = n_df
dic_13 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_13

description       = "Data from education-statistics"
name              = "df_Educ_foot_note"
file              = "education-statistics/EdStatsFootNote.csv"
n_df              = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol        = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_Educ_foot_note = n_df
dic_14 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_14

description    = "Data from education-statistics"
name           = "df_Educ_series"
file           = "education-statistics/EdStatsSeries.csv"
n_df           = pd.read_csv(path+file, sep=",", nrows = nRowsRead)
nRow, nCol     = n_df.shape
print("There are {} rows and {} columns in DataFrame {}.".format(nRow, nCol, name))
df_Educ_series = n_df
dic_15 = {'name': name, 'path': (path+file), 'nLines': nRow, 'nColomn': nCol, 'Description': description}
#dic_15


There are 613 rows and 4 columns in DataFrame df_Educ_country_series.
There are 241 rows and 32 columns in DataFrame df_Educ_country.
There are 1000 rows and 70 columns in DataFrame df_Educ_data.
There are 1000 rows and 5 columns in DataFrame df_Educ_foot_note.
There are 1000 rows and 21 columns in DataFrame df_Educ_series.


In [39]:
list_dic1 = [dic_1, dic_2, dic_3, dic_4, dic_5, dic_6, dic_8, dic_10, dic_11, dic_12, dic_13]
for dic in list_dic1:
    print(type(dic))


<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


In [45]:
#%xdel n_df
%who_ls dict


['dic_1',
 'dic_10',
 'dic_11',
 'dic_12',
 'dic_13',
 'dic_14',
 'dic_15',
 'dic_2',
 'dic_3',
 'dic_4',
 'dic_5',
 'dic_6',
 'dic_8']

----

In [ ]:
#%whos DataFrame
all_df = %who_ls DataFrame
all_df



### df_sas

### df_immigration

### df_indicator_dev

### df_temperature

### df_country

### df_country_series

### df_series

### df_data

### df_foot_note

### df_df_demograph

### df_airport_world

### df_airport_us

### df_airline_delay

In [ ]:
df_airline_delay.head(2)


In [ ]:
df_airline_delay.info()

In [ ]:
#len(df_airline_delay)

In [ ]:
df_airline_delay.columns

In [ ]:
#df_airline_delay.dtypes

In [ ]:
df_airline_delay.describe()

In [ ]:
#df_airline_delay.values

In [ ]:
df_airline_delay.describe(include=np.object)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.